In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [9]:
%cd /gdrive/My Drive/Colab Notebooks/AnnDl1Homework

/gdrive/My Drive/Colab Notebooks/AnnDl1Homework


### Import libraries

In [10]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.9.2


### Set seed for reproducibility

In [11]:
# Random seed for reproducibility
seed = 41

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Suppress warnings

In [12]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [14]:
cd data

/gdrive/MyDrive/Colab Notebooks/AnnDl1Homework/data


In [15]:
!ls

test  training	training_data_final  validation


In [16]:
# Download the data
dataset = '/gdrive/My Drive/Colab Notebooks/AnnDl1Homework/data/trainig_data_final'
training = '/gdrive/My Drive/Colab Notebooks/AnnDl1Homework/data/training'
validation = '/gdrive/My Drive/Colab Notebooks/AnnDl1Homework/data/validation'
test = '/gdrive/My Drive/Colab Notebooks/AnnDl1Homework/data/test'

In [ ]:
labels = {
    0:'species1',
    1:'species2',
    2:'species3',
    3:'species4',
    4:'species5',
    5:'species6',
    6:'species7',
    7:'species8'
}

### Process data

In [17]:
# Images are divided into folders, one for each class.
# If the images are organized in such a way, we can exploit the
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
test_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
valid_gen = train_data_gen.flow_from_directory(directory=validation,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
test_gen = train_data_gen.flow_from_directory(directory=test,
                                              target_size=(96,96),
                                              color_mode='rgb',
                                              classes=None, # can be set to labels
                                              class_mode='categorical',
                                              batch_size=32,
                                              shuffle=False,
                                              seed=seed
                                              )

Found 2701 images belonging to 8 classes.
Found 366 images belonging to 8 classes.
Found 475 images belonging to 8 classes.


In [18]:
# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=20,
                                        horizontal_flip= True,
                                        vertical_flip= True,
                                        brightness_range=(0.6,1.4),
                                        zoom_range=0.6,
                                        fill_mode='nearest',
                                        preprocessing_function = preprocess_input) # rescale value is multiplied to the image

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=training,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=32,
                                                       shuffle=True,
                                                       seed=seed)

Found 2701 images belonging to 8 classes.


### Models metadata

In [19]:
input_shape = (96, 96, 3)
batch_size = 32
epochs = 200

### Standard model

### Transfer learning

In [20]:
# Download and plot the VGG16 model
supernet = tfk.applications.EfficientNetV2B3(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3)
)
supernet.summary()

52606240/52606240 [==============================] - 3s 0us/step
Model: "efficientnetv2-b3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 96, 96, 3)    0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 96, 96, 3)    0           ['rescaling[0][0]']              
                                                                                                  
 stem_conv (Conv2D)             (None, 48, 48, 40)   1080        ['normalization[0][0]']          
                 

In [21]:
# Use the supernet as feature extractor
supernet.trainable = False

inputs = tfk.Input(shape=(96,96,3))
x = supernet(inputs)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    128, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    8,
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetv2-b3 (Function  (None, 3, 3, 1536)       12930622  
 al)                                                             
                                                                 
 Flattening (Flatten)        (None, 13824)             0         
                                                                 
 dropout (Dropout)           (None, 13824)             0         
                                                                 
 dense (Dense)               (None, 128)               1769600   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                             

In [22]:
# Train the model
tl_history = tl_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

Epoch 1/200
85/85 [==============================] - 1560s 18s/step - loss: 1.8662 - accuracy: 0.3251 - val_loss: 1.4442 - val_accuracy: 0.4672
Epoch 2/200
85/85 [==============================] - 13s 154ms/step - loss: 1.5535 - accuracy: 0.4165 - val_loss: 1.2985 - val_accuracy: 0.5273
Epoch 3/200
85/85 [==============================] - 13s 153ms/step - loss: 1.5153 - accuracy: 0.4347 - val_loss: 1.1710 - val_accuracy: 0.5956
Epoch 4/200
85/85 [==============================] - 14s 168ms/step - loss: 1.4396 - accuracy: 0.4713 - val_loss: 1.1251 - val_accuracy: 0.6175
Epoch 5/200
85/85 [==============================] - 13s 149ms/step - loss: 1.4095 - accuracy: 0.4798 - val_loss: 1.0721 - val_accuracy: 0.5765
Epoch 6/200
85/85 [==============================] - 13s 150ms/step - loss: 1.3982 - accuracy: 0.5006 - val_loss: 1.0252 - val_accuracy: 0.6093
Epoch 7/200
85/85 [==============================] - 13s 153ms/step - loss: 1.3343 - accuracy: 0.5017 - val_loss: 1.0651 - val_accuracy:

In [23]:
# Save the best model

tl_model.save('TransferLearningModel')
del tl_model

In [24]:
model_aug = tfk.models.load_model("TransferLearningModel")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with VGG")
print(model_aug_test_metrics)

15/15 [==============================] - 259s 18s/step - loss: 0.8078 - accuracy: 0.7263

Test metrics with VGG
{'loss': 0.8078048825263977, 'accuracy': 0.7263157963752747}


### Fine tuning

In [25]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModel')
ft_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetv2-b3 (Function  (None, 3, 3, 1536)       12930622  
 al)                                                             
                                                                 
 Flattening (Flatten)        (None, 13824)             0         
                                                                 
 dropout (Dropout)           (None, 13824)             0         
                                                                 
 dense (Dense)               (None, 128)               1769600   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                             

In [26]:
# Set all VGG layers to True
ft_model.get_layer('efficientnetv2-b3').trainable = True
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers):
   print(i, layer.name, layer.trainable)

0 input_1 True
1 rescaling True
2 normalization True
3 stem_conv True
4 stem_bn True
5 stem_activation True
6 block1a_project_conv True
7 block1a_project_bn True
8 block1a_project_activation True
9 block1b_project_conv True
10 block1b_project_bn True
11 block1b_project_activation True
12 block1b_drop True
13 block1b_add True
14 block2a_expand_conv True
15 block2a_expand_bn True
16 block2a_expand_activation True
17 block2a_project_conv True
18 block2a_project_bn True
19 block2b_expand_conv True
20 block2b_expand_bn True
21 block2b_expand_activation True
22 block2b_project_conv True
23 block2b_project_bn True
24 block2b_drop True
25 block2b_add True
26 block2c_expand_conv True
27 block2c_expand_bn True
28 block2c_expand_activation True
29 block2c_project_conv True
30 block2c_project_bn True
31 block2c_drop True
32 block2c_add True
33 block3a_expand_conv True
34 block3a_expand_bn True
35 block3a_expand_activation True
36 block3a_project_conv True
37 block3a_project_bn True
38 block3b_expa

In [27]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers[:190]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

0 input_1 False
1 rescaling False
2 normalization False
3 stem_conv False
4 stem_bn False
5 stem_activation False
6 block1a_project_conv False
7 block1a_project_bn False
8 block1a_project_activation False
9 block1b_project_conv False
10 block1b_project_bn False
11 block1b_project_activation False
12 block1b_drop False
13 block1b_add False
14 block2a_expand_conv False
15 block2a_expand_bn False
16 block2a_expand_activation False
17 block2a_project_conv False
18 block2a_project_bn False
19 block2b_expand_conv False
20 block2b_expand_bn False
21 block2b_expand_activation False
22 block2b_project_conv False
23 block2b_project_bn False
24 block2b_drop False
25 block2b_add False
26 block2c_expand_conv False
27 block2c_expand_bn False
28 block2c_expand_activation False
29 block2c_project_conv False
30 block2c_project_bn False
31 block2c_drop False
32 block2c_add False
33 block3a_expand_conv False
34 block3a_expand_bn False
35 block3a_expand_activation False
36 block3a_project_conv False
37 bl

In [28]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [29]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

Epoch 1/200
85/85 [==============================] - 30s 193ms/step - loss: 3.3975 - accuracy: 0.3536 - val_loss: 1.3103 - val_accuracy: 0.5410
Epoch 2/200
85/85 [==============================] - 14s 162ms/step - loss: 1.7912 - accuracy: 0.4350 - val_loss: 1.1428 - val_accuracy: 0.5820
Epoch 3/200
85/85 [==============================] - 14s 161ms/step - loss: 1.4652 - accuracy: 0.4869 - val_loss: 1.0945 - val_accuracy: 0.6093
Epoch 4/200
85/85 [==============================] - 14s 160ms/step - loss: 1.3905 - accuracy: 0.5102 - val_loss: 1.0425 - val_accuracy: 0.6175
Epoch 5/200
85/85 [==============================] - 15s 178ms/step - loss: 1.3090 - accuracy: 0.5257 - val_loss: 1.0076 - val_accuracy: 0.6366
Epoch 6/200
85/85 [==============================] - 14s 162ms/step - loss: 1.2783 - accuracy: 0.5376 - val_loss: 0.9775 - val_accuracy: 0.6503
Epoch 7/200
85/85 [==============================] - 14s 160ms/step - loss: 1.2421 - accuracy: 0.5553 - val_loss: 0.9470 - val_accuracy:

In [30]:
ft_model.save('FineTuningModel')
del ft_model

In [31]:
model_aug = tfk.models.load_model("FineTuningModel")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with VGG")
print(model_aug_test_metrics)

15/15 [==============================] - 5s 64ms/step - loss: 0.4107 - accuracy: 0.8674

Test metrics with VGG
{'loss': 0.41072043776512146, 'accuracy': 0.8673684000968933}
